In [1]:
import os 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import time 
import pickle
import warnings


from utils import *

from sensor_params import *

from models.training import *

from models.multi_step.unet1D_regressor import *
from models.multi_step.unet1D_regressor import get_model as get_unet1D

# from models.multi_step.unet1D_DIL_regressor import *
# from models.multi_step.unet1D_DIL_regressor import get_model as get_DIL_unet1D

from models.multi_step.LSTMVanilla import *
from models.multi_step.LSTMVanilla import get_model as get_LSTM

from models.multi_step.StackedLSTM import *
from models.multi_step.StackedLSTM import get_model as get_StackedLSTM

# from models.multi_step.unet1D_LSTM_regressor import *
# from models.multi_step.unet1D_LSTM_regressor import get_model as get_unet1DLSTM

# from models.multi_step.unet1D_nonCompres_regressor import *
# from models.multi_step.unet1D_nonCompres_regressor import get_model as get_unet1D_nonCompres

from evaluation.multi_step import *


# Dataset path 
DATABASE_PATH = r"C:\Users\aralmeida\OneDrive - Universidad de Las Palmas de Gran Canaria\Doctorado\Bases de datos\WARIFA\Mikael T1DM"

filename = "MIKAEL_data.json"

parent_directory = r"C:\Users\aralmeida\Downloads"
experiments_folder = r"\T1DM_pred_experiments" 

In [2]:
# Load data from sgv_data. If executed, comment the line and load the data from the pickle file
# data , basal_data_dict, blood_pressure_dict, bolus_data_dict, exercise_data_dict, carbohydrates_data_dict, pump_events_data_dict, sgv_data_dict, sleep_data_dict, smgb_data_dict, steps_data_dict, weight_dict  = extract_Mikael_data(DATABASE_PATH, filename, ONLY_CGM = True)
with open(DATABASE_PATH+'\CGM.pk1', 'rb') as f:
    sgv_data_dict = pickle.load(f)
os.chdir(parent_directory)

In [3]:
# Dataset generation parameters
N = 144
step = 1
PH = 60
pred_steps = round(PH/sensor_Mikael["SAMPLE_PERIOD"])
name = 'StackedLSTM-minmax-himar-rep'

In [4]:
X, Y, X_times, Y_times = get_CGM_X_Y_multistep(sgv_data_dict, sensor_Mikael, N, step, PH, experiments_folder, plot=False, verbose = 0)

Number of blocks is 335



In [5]:
# Min-max normalization
X_norm = (X - np.min(X))/(np.max(X) - np.min(X))
Y_norm = (Y - np.min(X))/(np.max(X) - np.min(X))

In [6]:
# Simple partition (imitating Himar work for comparison): X_train until 30/05/2021 and X_test from 31/05/2021
# Instances that include two days are removed
X_train = X_norm[np.where(X_times[:,N-1] <= pd.to_datetime('2021-05-31 00:00:00'))[0]]
Y_train = Y_norm[np.where((Y_times[:,0] < pd.to_datetime('2021-05-30 23:59:59')))[0]]
X_test = X_norm[np.where((X_times[:,N-1] > pd.to_datetime('2021-06-01 00:00:00')))[0]]
Y_test = Y[np.where((Y_times[:,0] > pd.to_datetime('2021-06-01 00:00:00')))[0]] # Left non-normalized to compute the metrics

print("X_train shape: ",str(X_train.shape))
print("Y_train shape: ",str(Y_train.shape))
print("X_test shape: ",str(X_test.shape))
print("Y_test shape: ",str(Y_test.shape))

X_train shape:  (113418, 144)
Y_train shape:  (113418, 12)
X_test shape:  (42797, 144)
Y_test shape:  (42797, 12)


In [7]:
# Parameters for training
# Best parameters for the model N 48
lr = 0.0001
batch_size = 2
epochs = 10

# Model hyperparameters
kernel_size = 3
stride = 1

# lr = 0.001
# batch_size = 2
# epochs = 10
# kernel_size = 3

In [ ]:
# Get model instance
################### CNNs/ UNETs ###################
model =  get_unet1D(sensor = sensor_Mikael,
                N=N,
                input_features = NUMBER_OF_INPUT_SIGNALS,
                tau=stride,
                kernel_size=kernel_size,
                PH=PH)
print(model.summary())
# model =  get_unet1D_nonCompres(N=N,
#                 input_features = NUMBER_OF_INPUT_SIGNALS,
#                 tau=tau,
#                 kernel_size=kernel_size,
#                 predicted_points=1)

# model =  get_DIL_unet1D(N=N,
#                 input_features = NUMBER_OF_INPUT_SIGNALS,
#                 tau=tau,
#                 kernel_size=kernel_size,
#                 dilation_rate=1,  
#                 predicted_points=1)

################### LSTMs ###################
model =  get_LSTM(sensor = sensor_Mikael,
                N=int(N),
                input_features = NUMBER_OF_INPUT_SIGNALS,
                PH=PH)

model =  get_StackedLSTM(sensor = sensor_Mikael,
                N=N,
                input_features = NUMBER_OF_INPUT_SIGNALS,
                PH=PH)


################### Hybryds ###################
# model =  get_unet1DLSTM(N=N,
#                 input_features = NUMBER_OF_INPUT_SIGNALS,
#                 tau=tau,
#                 kernel_size=kernel_size,
#                 predicted_points=1)

In [ ]:
train_model(model,
            X = X_train,
            Y = Y_train,
            N = N,
            stride = stride,             
            kernel_size = kernel_size,
            predicted_points = pred_steps, 
            sensor = sensor_Mikael,
            epochs = epochs,
            batch_size = batch_size,
            lr = lr,
            verbose = 1, 
            fold = name)

In [ ]:
model_evaluation(N, PH, name, X_test, Y_test, pred_steps, X) 